In [24]:
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
import kaggle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import featuretools as ft
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer, MultiLabelBinarizer


import os
import composeml as cp
import featuretools as ft
import dask.dataframe as dd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, recall_score
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
from dask.distributed import Client

### Importing and preparing the data

In [2]:
#Convert csv to DataFrame
oct19 = pd.read_csv("../data/2019-Oct.csv")
nov19 = pd.read_csv("../data/2019-Nov.csv")
dec19 = pd.read_csv("../data/2019-Dec.csv")
jan20 = pd.read_csv("../data/2020-Jan.csv")
feb20 = pd.read_csv("../data/2020-Feb.csv")

In [62]:
print(oct19.shape)
print(nov19.shape)
print(dec19.shape)
print(jan20.shape)
print(feb20.shape)
print(oct19.shape[0]+nov19.shape[0]+dec19.shape[0]+jan20.shape[0]+feb20.shape[0])

(4102283, 9)
(4635837, 9)
(3533286, 9)
(4264752, 9)
(4156682, 9)
20692840


In [63]:
ecom_df = pd.concat([oct19, nov19, dec19, jan20, feb20], axis=0)
ecom_df.head()
print(ecom_df.shape)

(20692840, 9)


# Data Cleaning

### NAs and how to deal with them

In [64]:
#8757117 nas is Brand
#4598 nas in user_session
#20339246 nas in category_code DROPPED

In [65]:
#NAs in Brand
ecom_df.brand.fillna(value="unknown", axis=0, inplace=True)

In [66]:
#NAs in user_session 
ecom_df.dropna(inplace=True)

In [67]:
ecom_df.category_code.isna().sum()

0

In [68]:
#Drop this column, as we will use category code instead
ecom_df.drop("category_id", axis=1, inplace = True)

In [69]:
#Get rid of negative prices
ecom_df = ecom_df[ecom_df['price']>= 0]

# Feature Engineering

purchase per session, group session, total of price added up for each purchase in session

### Time FE

In [70]:
ecom_df['event_time'] = pd.to_datetime(ecom_df['event_time'])

In [71]:
ecom_df['month'] = pd.DatetimeIndex(ecom_df['event_time']).month
ecom_df['year'] = pd.DatetimeIndex(ecom_df['event_time']).year
ecom_df['hour'] = pd.DatetimeIndex(ecom_df['event_time']).hour
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,hour
21,2019-10-01 00:00:53+00:00,view,5856191,appliances.environment.vacuum,runail,24.44,507355498,944c7e9b-40bd-4112-a05b-81e73f37e0c0,10,2019,0
264,2019-10-01 00:07:58+00:00,remove_from_cart,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
269,2019-10-01 00:08:03+00:00,remove_from_cart,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
274,2019-10-01 00:08:08+00:00,remove_from_cart,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
276,2019-10-01 00:08:10+00:00,remove_from_cart,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0


In [72]:
ecom_df['day'] = ecom_df['event_time'].dt.dayofweek
ecom_df['day'] = ecom_df['day'].replace({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})

In [73]:
when_df = pd.DataFrame({'hour':range(1, 25)})
bins = [0,6,12,18,24]
labels = ['Night', 'Morning','Afternoon','Evening']
when_df['when'] = pd.cut(when_df['hour'], bins=bins, labels=labels, include_lowest=True)

ecom_df['time_of_day'] = when_df['when']

In [75]:
ecom_df.time_of_day.value_counts()

Evening      1
Afternoon    0
Morning      0
Night        0
Name: time_of_day, dtype: int64

In [76]:
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,hour,day,time_of_day
21,2019-10-01 00:00:53+00:00,view,5856191,appliances.environment.vacuum,runail,24.44,507355498,944c7e9b-40bd-4112-a05b-81e73f37e0c0,10,2019,0,Tuesday,Evening
264,2019-10-01 00:07:58+00:00,remove_from_cart,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0,Tuesday,NaN
269,2019-10-01 00:08:03+00:00,remove_from_cart,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0,Tuesday,NaN
274,2019-10-01 00:08:08+00:00,remove_from_cart,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0,Tuesday,NaN
276,2019-10-01 00:08:10+00:00,remove_from_cart,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0,Tuesday,NaN


In [77]:
ecom_df["current_session_clicks"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

DataError: No numeric types to aggregate

In [68]:
#need to turn the features into numeric for bellow to work
ecom_df["current_session_event_types"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

DataError: No numeric types to aggregate

# Labeling

In [ ]:
encoder = LabelBinarizer()
etype_labels = pd.DataFrame(encoder.fit_transform(ecom_df['event_type']), columns=encoder.classes_, index=ecom_df.index)
etype_labels.head()

In [ ]:
encoder = LabelBinarizer()
brand_labels = pd.DataFrame(encoder.fit_transform(ecom_df['brand']), columns=encoder.classes_, index=ecom_df.index)
brand_labels.head()

In [ ]:
encoder = LabelBinarizer()
productid_labels = pd.DataFrame(encoder.fit_transform(ecom_df['product_id']), columns=encoder.classes_, index=ecom_df.index)
productid_labels.head()

In [26]:
ohe = OneHotEncoder()
catco_labels = pd.DataFrame(ohe.fit_transform(ecom_df['category_code']), columns=encoder.classes_, index=ecom_df.index)
catco_labels.head()

ValueError: Expected 2D array, got 1D array instead:
array=['appliances.environment.vacuum' 'stationery.cartrige'
 'stationery.cartrige' ... 'furniture.living_room.cabinet'
 'appliances.personal.massager' 'appliances.environment.vacuum'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
#Join our Labeled Columns to our main DataFrame
ecom_df = ecom_df.join(etype_labels)
ecom_df = ecom_df.join(brand_labels)
ecom_df = ecom_df.join(productid_labels)
ecom_df = ecom_df.join(catid_labels)
ecom_df.head()

### Purchases_df

Created a dataframe of just purchases to do simplier features

In [ ]:
purchases_df = ecom_df.loc[ecom_df.event_type=='purchase']
purchases_df.shape

In [ ]:
#add column that adds total value of purchases for that user_session
purchases_df["session_cumulative_purchases"] = purchases_df.groupby(purchases_df.user_session)["price"].cumsum()
#ecom_df['duplicate'] = ecom_df.groupby(cols)['user_id'].transform('size')
#can use cumsum to generate up to this point sums by user ID
purchases_df["lifetime_cumulative_purchases"] = purchases_df.groupby(purchases_df.user_id)["price"].cumsum()
#purchases from session
purchases_df["purchases_in_session"] = purchases_df.groupby(purchases_df.user_id)["user_session"].transform('count')
#add event_type numericall
purchases_df["event_type"]=0
purchases_df.head()

### Removed_df

Created a dataframe of just removed to do simplier features

In [ ]:
removed_df = ecom_df.loc[ecom_df.event_type=="remove_from_cart"]
removed_df["event_type"]=1
removed_df.head()

# LETS TRY AN FSM SHRUGS

10/17 10pm update theres an issue with the merge thats causing the classes to become a single class. Will fix this tomorrow zZz

In [ ]:
temp_df = ecom_df[ecom_df.event_type != 'view']
temp_df = temp_df[temp_df.event_type != 'cart']
print(temp_df.event_type.value_counts())
print(temp_df.shape)

In [ ]:
#purchases_df.drop("event_type", axis=1, inplace=True)
#removed_df.drop("event_type", axis=1, inplace=True)
dataframes=[purchases_df, removed_df]
fsm = pd.concat(dataframes)

In [ ]:
#quick nans fix for missing columns from removed_df
fsm['session_cumulative_purchases'] = fsm['session_cumulative_purchases'].fillna(0)
fsm['purchases_in_session'] = fsm['purchases_in_session'].fillna(0)
fsm['lifetime_cumulative_purchases'] = fsm['lifetime_cumulative_purchases'].fillna(0)
fsm.sample(10)

In [ ]:
#HERE WE ARE LOOSING ALL MY CLASSES
le = preprocessing.LabelEncoder()

le.fit(ecom_df['event_type'])

ecom_df['event_type']=le.transform(ecom_df['event_type'])

ecom_df.event_type.value_counts()

# Models

### FSM

This is a proof of concept not I will spend my time feature engineering to make this model better. I have started some of it above.

In [69]:
ecom_df = ecom_df[ecom_df.event_type != 'view']
ecom_df = ecom_df[ecom_df.event_type != 'cart']
ecom_df.sample()


,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,cart,purchase,...,1487580013053083824,1783999067181810204,1921723506584715388,2007399943458784057,2018287324474901238,2022622168218599898,2193074740619379535,2193074740686488401,2235524499216793855,purchase_this_session
2794973,2020-01-22 06:03:14 UTC,remove_from_cart,5748490,1487580006350586771,unknown,26.97,473793381,994de488-461c-4c4a-b7a1-8122aa7ec4c5,1,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
#"total_visits", "total_purchases_session"
X = ecom_df.drop(["purchase", "user_id", "product_id", "category_id", "cart", "purchase", "remove_from_cart", "view", "event_time", "event_type", "brand", "user_session"], axis=1)
y = ecom_df.event_type

In [83]:
X.sample(20)
#why is their cumulative purchases in session in where nothing was bought

,price,babyliss,benovy,concept,cutrin,depilflax,dewal,domix,emil,entity,...,1487580013053083824,1783999067181810204,1921723506584715388,2007399943458784057,2018287324474901238,2022622168218599898,2193074740619379535,2193074740686488401,2235524499216793855,purchase_this_session
1481307,1.27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2584446,29.71,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
449163,3.02,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2243756,1.98,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3342540,5.62,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3421727,4.16,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
710257,5.24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1746254,1.11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2156724,79.21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2171219,9.52,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
import lightgbm as lgb

In [74]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_train, y_train)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_train)
test_preds_lgbm = lgbm.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

                  precision    recall  f1-score   support

        purchase       0.94      0.92      0.93      7011
remove_from_cart       0.96      0.97      0.97     16036

        accuracy                           0.96     23047
       macro avg       0.95      0.94      0.95     23047
    weighted avg       0.96      0.96      0.96     23047



# Visuals

### Graph 1

In [30]:
user_df = ecom_df[['user_id','event_type','price']]

In [33]:
graph_df = user_df[['user_id','price']]
graph_df = graph_df.groupby(['user_id'])['price'].sum().reset_index()
graph_df = graph_df.sort_values(by=['price'],ascending=False)
graph_df = graph_df.rename(columns={'price':'spent'})

In [37]:
price_list = graph_df['spent'].to_list()
plt.hist(x= price_list,bins=100,log=True, color='green')
plt.xlabel('Spent amount')
plt.ylabel('Customer Count')
plt.title('Customer Spending')
plt.savefig("Customer_Spend")

### Graph 2

In [47]:
# Daily visitors number
date = ecom_df.loc[:,['event_time','user_id']]
visitor_by_date = date.groupby(['event_time'])['user_id'].nunique()
visitor_by_date

event_time
2019-10-01 00:00:53+00:00    1
2019-10-01 00:07:58+00:00    1
2019-10-01 00:08:03+00:00    1
2019-10-01 00:08:08+00:00    1
2019-10-01 00:08:10+00:00    1
                            ..
2020-02-29 23:47:54+00:00    1
2020-02-29 23:51:55+00:00    1
2020-02-29 23:54:28+00:00    1
2020-02-29 23:54:43+00:00    1
2020-02-29 23:54:44+00:00    1
Name: user_id, Length: 330514, dtype: int64